In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests


In [2]:
#連上Finmind取得及時的數據
url = "https://api.finmindtrade.com/api/v4/login"
parload = {
    "user_id": "user_id",
    "password": "password",
}
data = requests.post(url, data=parload)
data=data.json()
my_token=data['token']

KeyError: 'token'

In [ ]:
url = "https://api.finmindtrade.com/api/v4/data"
parameter = {
    "dataset": "TaiwanStockPrice",
    "data_id": "2330",
    "start_date": "2020-01-01",
    "end_date": "2021-06-03",
    "token":my_token # 參考登入，獲取金鑰
}
resp = requests.get(url, params=parameter)
data = resp.json()
df = pd.DataFrame(data["data"])
df

In [ ]:
#取得個股的融資卷增減量(張) (data2)
parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "data_id": "2603",
    "start_date": "2020-01-01",
    "token": my_token, # 參考登入，獲取金鑰
}
data2 = requests.get(url, params=parameter).json()
data2 = pd.DataFrame(data2['data'])
data2=data2[["MarginPurchaseTodayBalance","MarginPurchaseYesterdayBalance","ShortSaleTodayBalance","ShortSaleYesterdayBalance"]]
data2["MarginPurchaseVariation"]=data2["MarginPurchaseTodayBalance"]-data2["MarginPurchaseYesterdayBalance"]
data2["ShortSaleVariation"]=data2["ShortSaleTodayBalance"]-data2["ShortSaleYesterdayBalance"]
data2

data2=data2.drop(343,axis=0)
data2

In [ ]:
#取得個股的股利,本益比,淨值比(data3)
parameter = {
    "dataset": "TaiwanStockPER",
    "data_id": "2330",
    "start_date": "2020-01-01",
    "token": my_token, # 參考登入，獲取金鑰
}
data3 = requests.get(url, params=parameter).json()
data3 = pd.DataFrame(data3['data'])
data3=data3.drop(["date","stock_id"],axis=1)

data3=data3.drop(343,axis=0)
data3

In [ ]:
#取得大盤的融資餘額(百萬)以及融卷張數 (data4)
parameter = {
    "dataset": "TaiwanStockTotalMarginPurchaseShortSale",
    "start_date": "2020-01-01",
    "token": my_token, # 參考登入，獲取金鑰
}
data4 = requests.get(url, params=parameter).json()
data4 = pd.DataFrame(data4['data'])[["TodayBalance","name"]]
data4=data4[~data4['name'].isin(["MarginPurchase"])]
MarginPurchaseMoney=data4[data4["name"]=="MarginPurchaseMoney"].drop(columns="name")
MarginPurchaseMoney.rename(columns={"TodayBalance":"MarginPurchaseMoney"})
MarginPurchaseMoney=MarginPurchaseMoney["TodayBalance"]/10000000
MarginPurchaseMoney=MarginPurchaseMoney.to_frame().rename(columns={"TodayBalance":"MarginPurchaseMoney"})
ShortSale=data4[data4["name"]=="ShortSale"].drop(columns="name")
ShortSale=ShortSale.rename(columns={"TodayBalance":"Shortsale"})
mod=list(range(len(df)))
MarginPurchaseMoney=MarginPurchaseMoney.set_axis(mod)
ShortSale=ShortSale.set_axis(mod)
data4=MarginPurchaseMoney.join(ShortSale)
data4

data4=data4.drop(343,axis=0)
data4

In [ ]:
#大盤指數,與漲跌點數
data5 = pd.read_csv('TAIEX.csv')
data5=data5.iloc[::-1]
mod=list(range(len(df)))
data5=data5.set_axis(mod).drop(["日期","報酬指數值",'漲跌百分比'],axis=1)
data5.columns = ['大盤指數','漲跌點數']
data5

In [ ]:
#將多張dataframe合併
df=df.join(data2)
df=df.join(data3)
df=df.join(data4)
df=df.join(data5)
df

In [ ]:
df.columns=["日期","股票編號","當日成交量","當日成交金額","開盤價","最高價","最低價","收盤價","買賣價差","周轉率","當日融資","昨日融資","當日融卷","昨日融卷","融卷變化","融資變化","殖利率","本益比","股價淨值比","大盤融資餘額(百萬)","大盤融卷量","大盤指數","大盤漲跌"]

In [ ]:
for column in df:
    print(column)

In [ ]:
df